### Generate a list of Saturday between defined period

In [1]:
# import dependencies
import numpy as np
import datetime
import requests
from pprint import pprint
import pandas as pd
import os
import json

In [2]:
# define a function to generate the closest saturday of our desired date
def getsaturday(date):
    "@date: given date, in format '2013-05-25'"
    d1 = datetime.datetime.strptime(date, '%Y-%m-%d')
    t = datetime.timedelta((7 + 5 - d1.weekday()) % 7) # 5 is saturday
    next_saturday = d1 + t
    return next_saturday

In [3]:
# function to calculate the range for the for loop to generate list
def getdaterange(startdate, lastdate):
    "@startdate & lastdate: given date, in format '2013-05-25'"
    d1 = getsaturday(startdate)
    d2 = datetime.datetime.strptime(lastdate,'%Y-%m-%d')
    date_range = (d2 - d1).days/7
    return int(date_range)+1

In [4]:
def getdatelist(startdate, lastdate):
    "@startdate & lastdate: given date, in format '2013-05-25'"
    "@generate a list of saturday between the date range"
    # Create a list to store all saturdays
    date_list = []
    #date1 = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    #date2 = datetime.datetime.strptime(last_date, '%Y-%m-%d')
    d1 = getsaturday(start_date)
    d2 = getsaturday(last_date)
    n = getdaterange(start_date, last_date)
    aweekago = datetime.timedelta(days=7)
    date_list.append(d2)
    date = d2 # define date as the last saturday of the datelist
    for i in range(n):
        new_date = date - aweekago
        date_list.append(new_date)
        date = new_date
    return date_list

## Last.fm API to obtain list of artist & song name

In [5]:
# use last.fm API to generate a list of tags
API_KEY = '395ba12d921dee55ad263b4c8b53dfc7'
url = 'http://ws.audioscrobbler.com/2.0/'
tags_url = f'{url}?method=chart.gettoptags&api_key={API_KEY}&format=json'
response = requests.get(tags_url)

In [6]:
tags_df = pd.DataFrame(response.json()['tags']['tag'])
tags_df = tags_df.drop(columns=['streamable','wiki','url'])
tags_df['reach'] = pd.to_numeric(tags_df['reach'], errors='raise', downcast='integer')
tags_df = tags_df.sort_values(by=['reach'],ascending=False)
tags_df = tags_df.set_index('name',drop=True)
tags_df.head()

,reach,taggings
name,,
rock,393249,3934336
alternative,260111,2068031
electronic,251527,2334240
indie,251262,1994570
pop,222668,1929720


In [11]:
def gettoptracks(pages):
    # Create a dataframe
    columns = ['rank','artist_name','song_title','genres']
    df = pd.DataFrame(columns=columns)
    # define key variables
    n=pages
    url = 'http://ws.audioscrobbler.com/2.0/'
    tag_name = 'pop'
    for page in range(n):
        tracksbytag_url = f'{url}?method=tag.gettoptracks&tag={tag_name}&api_key={API_KEY}&format=json&page={page}'
        response = requests.get(tracksbytag_url)
        data = response.json()
        tracks = data['tracks']['track']
        for track in tracks:
            try:
                genres = tag_name
                rank = track['@attr']['rank']
                artist_name = track['artist']['name']
                song_name = track['name']
            except KeyError:
                rank = 'Not Available'
                artist_name = 'Not Available'
                song_name = 'Not Available'
            df = df.append({'rank':rank,
                          'artist_name':artist_name,
                          'song_title':song_name,
                          'genres':genres}, ignore_index=True)
    return df

In [12]:
tracks = gettoptracks(30)

In [13]:
tracks

,rank,artist_name,song_title,genres
0,1,Oasis,Wonderwall,pop
1,2,The Killers,Somebody Told Me,pop
2,3,Coldplay,Viva la Vida,pop
3,4,Coldplay,Clocks,pop
4,5,Coldplay,The Scientist,pop
5,6,MGMT,Kids,pop
6,7,Coldplay,Yellow,pop
7,8,Coldplay,Fix You,pop
8,9,Gorillaz,Feel Good Inc.,pop
9,10,Lady Gaga,Poker Face,pop
